<div style="background:#E9FFF6; color:#440404; padding:8px; border-radius: 4px; text-align: center; font-weight: 500;">IFQ619 - Data Analytics for Strategic Decision Makers (2024)</div>

# IFQ619 :: C1-UnstructuredAnalytics

For this session, the focus will be on analysis of unstructured text. However, the thinking required is similar to approaches to analysing images, video, sound and other unstructured data. Primarily, the analysis is based on the notion that there are useful patterns in the unstructured data which can be obtained mathematically. By converting the data to a mathematical structure, various algorithms can be applied to the structure with the aim of identifying patterns. 

In the case of the `topic modelling` approaches below, many of the techniques are *probabilistic* - that is they mathematically identify the *likelihood* that a feature might be important. Thus, they are never 100% accurate, and their use needs to be mediated by a more pragmatic *useful or not* approach, rather than *right or wrong*.

In [154]:
# Import the necessary libraries
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer, ENGLISH_STOP_WORDS
from sklearn.decomposition import LatentDirichletAllocation, NMF
import pandas as pd
import json
import random
from sklearn.cluster import KMeans

### Accessing the data via The Guardian API

See the `Accessing_the_Guardian_API.ipynb` notebook file for details on getting the data. **Note:** This approach may be used for additional data for Assignment 2.

### Read in pre-saved data

To save time, we're loading in pre-saved data that was fetched using the Guardian API.

In [155]:
# Load the data - articles from The Guardian about the war in Ukraine
file_path = "data/"
file_name = "qld_fund_articles.json"

with open(f"{file_path}{file_name}",'r', encoding='utf-8') as fp:
    articles = json.load(fp)

print(f"Loaded {len(articles)} articles from {file_name}")

Loaded 459 articles from qld_fund_articles.json


Each dictionary entry includes the *title [date]* as `key` and the *body text* from the article as `value`.

In [156]:
article1 = list(articles.items())[0]
print("Key:",article1[0])
print("Value:",article1[1][:300],"...") # Just show first 300 characters

Key: Queensland budget 2024: the biggest winners and losers [2024-06-11T04:38:46Z]
Value: The Queensland government’s pre-election budget is aimed squarely at the hip pocket. 
This year’s budget is built around a series of cost-of-living handouts for the “ordinary” Queenslander, including cheaper public transport and power. 
Here is a breakdown of the winners and losers in Queensland’s 2 ...


So the values gives us a list of documents that we can analyse.

In [157]:
# Get a list of documents
documents = list(articles.values())

# View first 400 characters of the 1st document
documents[0][:400]

'The Queensland government’s pre-election budget is aimed squarely at the hip pocket. \nThis year’s budget is built around a series of cost-of-living handouts for the “ordinary” Queenslander, including cheaper public transport and power. \nHere is a breakdown of the winners and losers in Queensland’s 2024 state budget. \nWinners \nHouseholds, commuters and first home buyers \nThe premier, Steven Miles, '

### Term Count 

**Finding important terms by the frequency of their occurance**

Using `CountVectorizer` create a `vector` for each document where the dimensionality of the vector is the `vocabulary` (all terms in the collection), and the value of each component is the number of times that the `term` occurs in the document.

All of these analyses, approach the document as a [Bag of Words](https://en.wikipedia.org/wiki/Bag-of-words_model) model. In this approach, the order of the words don't matter. A popular approach that takes into account order is [Word embedding](https://en.wikipedia.org/wiki/Word_embedding). This session does not explore word embedding.

In [158]:
# AYOOOOOO WE DOING A LEMMATISING FUNCTION
# https://jonathansoma.com/lede/image-and-sound/text-analysis/text-analysis-word-counting-lemmatizing-and-tf-idf/

import spacy
nlp = spacy.load("en_core_web_sm")
# Source: spaCy is a dream, but a dream where sometimes your legs won’t move right and you can’t read text. But sometimes you can fly! So yes, as always, ups and downs
# Morgan: I hate this
# python -m spacy download en
# python -m pip install spacy

def lemmatize(text):
    doc = nlp(text)
    # Turn it into tokens, ignoring the punctuation
    tokens = [token for token in doc if not token.is_punct]
    # Convert those tokens into lemmas, EXCEPT the pronouns, we'll keep those.
    lemmas = [token.lemma_ if token.pos_ != 'PRON' else token.orth_ for token in tokens]
    return lemmas

In [159]:
# Only count terms that in maximum of 75% of documents, and a minimum of 2 documents. 
# Count a maximum of 10000 terms, and remove common english stop words
StopWords = list(ENGLISH_STOP_WORDS.union(["budget", "funding", "Monday","Tuesday","Wednesday","Thursday","Friday", "Saturday", "Sunday","nbsp", "\n", "|", "\n ","government", "$", "year", "state", "australian", "labor", "m", "new", "need", "increase"]))
import re

#count_vectorizer = CountVectorizer(preprocessor=lambda x: x.replace('(\D+)', "", regex = True))
#count_vectorizer = CountVectorizer(preprocessor=lambda x: re.sub(r"([\d\.])+", '"NUM'", x.lower()),
count_vectorizer = CountVectorizer(
                                   max_df=0.85,min_df=2,max_features=10000,stop_words=StopWords, tokenizer=lemmatize)
count_dt_matrix = count_vectorizer.fit_transform(articles.values())



c:\Users\Morgan\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\feature_extraction\text.py:521: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
c:\Users\Morgan\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\feature_extraction\text.py:406: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['far', 'friday', 'make', 'monday', 'saturday', 'sunday', 'thursday', 'tuesday', 'wednesday', 'whereaft'] not in stop_words.
  warnings.warn(


**I HAVE ADDED MY OWN ADDITIONAL STOP WORDS HERE FOR NUMBERS**
NOt sufficient, referred to https://stackoverflow.com/questions/43216530/adding-numbers-to-stop-words-to-scikit-learns-countvectorizer

In [160]:
# Take a look at the vector for the first document
doc001_vector = count_dt_matrix.toarray()[0]
doc001_vector

array([1, 1, 0, ..., 0, 0, 0], dtype=int64)

In [161]:
# Get the 1000 terms identified during the vectorization process
feature_names = count_vectorizer.get_feature_names_out()
feature_names

array(['\n \n', '\n \n ', '\n \n \n ', ..., '“the', '”the', '️'],
      dtype=object)

In [162]:
# Look at how the counts match up to the terms (for the 1st doc)
#doc001_term_counts = list(zip(feature_names,doc001_vector[0])) 
#doc001_term_counts

In [163]:
# Take a look at the vocabulary which shows the total counts for whole collection
count_vectorizer.vocabulary_

{'queensland': 5822,
 'pre': 5593,
 'election': 2748,
 'aim': 805,
 'squarely': 6795,
 'hip': 3685,
 'pocket': 5515,
 'build': 1443,
 'series': 6482,
 'cost': 2100,
 'live': 4444,
 'handout': 3578,
 'ordinary': 5166,
 'queenslander': 5823,
 'include': 3871,
 'cheap': 1693,
 'public': 5776,
 'transport': 7346,
 'power': 5581,
 'breakdown': 1386,
 'winner': 7871,
 'loser': 4491,
 '2024': 255,
 'household': 3756,
 'commuter': 1880,
 'home': 3710,
 'buyer': 1490,
 'premier': 5614,
 'steven': 6874,
 'mile': 4757,
 'treasurer': 7356,
 'cameron': 1518,
 'dick': 2453,
 'hand': 3571,
 'concession': 1933,
 '31': 351,
 'great': 3492,
 '\n \n  ': 4,
 '50c': 458,
 'fare': 3074,
 '1,000': 53,
 'energy': 2817,
 'rebate': 5924,
 'favour': 3093,
 'surplus': 7032,
 'spend': 6754,
 ' \n \n': 26,
 'headline': 3622,
 'grab': 3463,
 'plan': 5481,
 'month': 4844,
 'trial': 7367,
 'mean': 4661,
 '20': 225,
 'discount': 2515,
 'car': 1556,
 'registration': 6016,
 'freeze': 3282,
 'fee': 3105,
 'charge': 1678,


In [164]:
count_vectorizer.vocabulary_["regional"]

6013

#### Display matrix in dataframe

Take the term count matrix and display in a dataframe to make visible the structure


In [165]:
count_dt_matrix.toarray()

array([[1, 1, 0, ..., 0, 0, 0],
       [1, 1, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [166]:
# Create a new dataframe with the matrix - use titles for the index and terms for the columns
count_df = pd.DataFrame(count_dt_matrix.toarray(), index=articles.keys(), columns=feature_names)
count_df

,\n \n,\n \n,\n \n \n,\n \n \n,\n \n,\n \n \n \n,\n \n \n,\n \n \n \n,\n \n \n,\n \n \n \n \n \n,...,à,’,’d,’ll,’m,’re,’ve,“the,”the,️
Queensland budget 2024: the biggest winners and losers [2024-06-11T04:38:46Z],1,1,0,0,3,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
"Queensland state budget 2024: 50c fares and $1,000 energy rebates favoured over surplus in pre-election spend [2024-06-11T04:00:44Z]",1,1,0,0,2,0,0,0,0,0,...,0,1,0,0,0,0,2,0,0,0
Queensland officially drought-free for first time since 2013 [2024-05-14T15:00:24Z],0,0,0,0,0,0,0,0,0,0,...,0,8,1,1,0,1,0,0,0,0
"With discounts galore, Labor’s 2024 Queensland budget has all the subtlety of a ‘going out of business’ sale [2024-06-11T07:00:37Z]",1,1,0,0,1,0,0,0,0,0,...,0,3,0,0,0,0,0,0,0,0
CFMEU administrator moves for ‘clean sweep’ of union super fund directors at Cbus [2024-08-28T08:51:22Z],1,1,0,0,1,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Most of Australia’s nuclear waste comes from Lucas Heights – should it stay there? [2022-10-16T16:30:40Z],0,0,0,0,0,0,0,0,0,0,...,0,4,0,0,0,1,0,0,0,0
Lidia Thorpe quits Greens party to pursue black sovereignty [2023-02-06T02:15:16Z],1,1,0,0,2,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Australia to provide ‘lethal aid’ to Ukraine as it fights invading Russian troops [2022-02-27T05:26:57Z],0,0,0,0,0,0,0,0,0,0,...,0,2,0,3,0,0,1,0,0,0
Environmental policies are taking a back seat in the SA election. Should voters be worried? [2022-03-18T01:19:56Z],0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,1,1,0,0,0


By selecting a row from the dataframe and sorting the values (counts), we can identify the top 10 terms

#### Create a top10 terms dataframe

Using the index from the documents, create a dataframe that can hold the top10 terms for each document. We also include columns for our other analysis (tfidf, lda, nmf)

In [167]:
# Create a dataframe to hold top terms for each analysis type
terms_df = pd.DataFrame(index=count_df.index,columns=['count','tfidf','lda','nmf'])
terms_df

,count,tfidf,lda,nmf
Queensland budget 2024: the biggest winners and losers [2024-06-11T04:38:46Z],NaN,NaN,NaN,NaN
"Queensland state budget 2024: 50c fares and $1,000 energy rebates favoured over surplus in pre-election spend [2024-06-11T04:00:44Z]",NaN,NaN,NaN,NaN
Queensland officially drought-free for first time since 2013 [2024-05-14T15:00:24Z],NaN,NaN,NaN,NaN
"With discounts galore, Labor’s 2024 Queensland budget has all the subtlety of a ‘going out of business’ sale [2024-06-11T07:00:37Z]",NaN,NaN,NaN,NaN
CFMEU administrator moves for ‘clean sweep’ of union super fund directors at Cbus [2024-08-28T08:51:22Z],NaN,NaN,NaN,NaN
...,...,...,...,...
Most of Australia’s nuclear waste comes from Lucas Heights – should it stay there? [2022-10-16T16:30:40Z],NaN,NaN,NaN,NaN
Lidia Thorpe quits Greens party to pursue black sovereignty [2023-02-06T02:15:16Z],NaN,NaN,NaN,NaN
Australia to provide ‘lethal aid’ to Ukraine as it fights invading Russian troops [2022-02-27T05:26:57Z],NaN,NaN,NaN,NaN
Environmental policies are taking a back seat in the SA election. Should voters be worried? [2022-03-18T01:19:56Z],NaN,NaN,NaN,NaN


Populate the count column with data created by the count vectorizer.

In [168]:
#For each doc, get the 10 columns with the largest counts
for idx in terms_df.index:
    counts = dict(count_df.loc[idx].sort_values(ascending=False).head(10))
    #print(counts)
    terms_df.at[idx,'count'] = list(counts.keys()) # Just the list of terms

terms_df

,count,tfidf,lda,nmf
Queensland budget 2024: the biggest winners and losers [2024-06-11T04:38:46Z],"[queensland, include, public, spend, election,...",NaN,NaN,NaN
"Queensland state budget 2024: 50c fares and $1,000 energy rebates favoured over surplus in pre-election spend [2024-06-11T04:00:44Z]","[dick, measure, cost, deficit, queensland, mak...",NaN,NaN,NaN
Queensland officially drought-free for first time since 2013 [2024-05-14T15:00:24Z],"[drought, queensland, ’, rain, country, farmer...",NaN,NaN,NaN
"With discounts galore, Labor’s 2024 Queensland budget has all the subtlety of a ‘going out of business’ sale [2024-06-11T07:00:37Z]","[election, discount, dick, cost, living, measu...",NaN,NaN,NaN
CFMEU administrator moves for ‘clean sweep’ of union super fund directors at Cbus [2024-08-28T08:51:22Z],"[union, construction, cfmeu, industry, secreta...",NaN,NaN,NaN
...,...,...,...,...
Most of Australia’s nuclear waste comes from Lucas Heights – should it stay there? [2022-10-16T16:30:40Z],"[waste, facility, nuclear, estimate, storage, ...",NaN,NaN,NaN
Lidia Thorpe quits Greens party to pursue black sovereignty [2023-02-06T02:15:16Z],"[thorpe, green, voice, party, bandt, support, ...",NaN,NaN,NaN
Australia to provide ‘lethal aid’ to Ukraine as it fights invading Russian troops [2022-02-27T05:26:57Z],"[ukraine, russia, russian, china, support, min...",NaN,NaN,NaN
Environmental policies are taking a back seat in the SA election. Should voters be worried? [2022-03-18T01:19:56Z],"[party, south, election, liberal, climate, ene...",NaN,NaN,NaN


In [169]:
# Sample 5 random articles
samples = random.sample(range(0,len(terms_df)),5)

for sample in samples:
    doc = count_df.iloc[sample]
    top_terms = dict(count_df.iloc[sample].sort_values(ascending=False).head(10))
    print(f"[{sample}] {doc.name}")
    print("\t- Top terms:",top_terms)

[27] Morning Mail: revolt in WA Labor; footage shows anguish of children in custody; pressure grows on Biden [2024-07-17T20:44:47Z]
	- Top terms: {'child': 6, 'house': 5, 'queensland': 5, 'watch': 5, '\n \n': 4, 'morning': 4, 'election': 4, 'win': 4, '\n \n ': 4, 'condition': 3}
[21] Morning Mail: budget’s $9.3bn surplus, BHP’s $65bn raid rejected, Trump’s ex-lawyer takes the stand [2024-05-13T20:59:30Z]
	- Top terms: {'queensland': 6, '\n \n': 4, 'tell': 4, 'morning': 4, '\n \n ': 4, 'day': 4, 'allege': 3, 'picture': 3, 'medium': 3, 'mining': 3}
[275] It’s so frustrating that an election brings funding to north Queensland after years of neglect | Rachael Coco  [2022-04-10T17:30:27Z]
	- Top terms: {'local': 10, 'support': 7, 'home': 6, 'exist': 5, 'benefit': 4, 'little': 4, 'issue': 4, 'business': 4, 'industry': 4, '  ': 4}
[434] I am a Medibank customer. Am I affected by the cyber-attack? What can I do to protect myself? [2022-11-11T06:08:21Z]
	- Top terms: {'customer': 24, 'medibank'

### Term Frequency / Inverse Document Frequency (TF/IDF)

**Finding terms that are very common in a document, but less common in the whole collection**

The [TF/IDF](https://en.wikipedia.org/wiki/Tf–idf) algorithm takes the term frequencies for a document and divides them by the frequencies of the terms in the whole collection.


In [170]:
# Only count terms that in maximum of 75% of documents, and a minimum of 2 documents. 
# Count a maximum of 10000 terms, and remove common english stop words
tfidf_vectorizer = TfidfVectorizer(
    max_df=0.85, min_df=2, max_features=10000, stop_words="english"
)

In [171]:
# Get the document vectors
tfidf_dt_matrix = tfidf_vectorizer.fit_transform(articles.values())

# Display the vector for the first document
tfidf_dt_matrix.toarray()[0]

array([0.06887436, 0.        , 0.        , ..., 0.        , 0.        ,
       0.        ])

#### Display matrix in dataframe

In [172]:
tfidf_df = pd.DataFrame(tfidf_dt_matrix.toarray(), index=articles.keys(), columns=tfidf_vectorizer.get_feature_names_out())
tfidf_df

,000,000km,060,065,069,072,08,080,0808,09,...,zealand,zelenskiy,zero,zimmerman,zoe,zombie,zone,zones,zonfrillo,zuckerberg
Queensland budget 2024: the biggest winners and losers [2024-06-11T04:38:46Z],0.068874,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"Queensland state budget 2024: 50c fares and $1,000 energy rebates favoured over surplus in pre-election spend [2024-06-11T04:00:44Z]",0.041327,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Queensland officially drought-free for first time since 2013 [2024-05-14T15:00:24Z],0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"With discounts galore, Labor’s 2024 Queensland budget has all the subtlety of a ‘going out of business’ sale [2024-06-11T07:00:37Z]",0.028481,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
CFMEU administrator moves for ‘clean sweep’ of union super fund directors at Cbus [2024-08-28T08:51:22Z],0.013087,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Most of Australia’s nuclear waste comes from Lucas Heights – should it stay there? [2022-10-16T16:30:40Z],0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Lidia Thorpe quits Greens party to pursue black sovereignty [2023-02-06T02:15:16Z],0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Australia to provide ‘lethal aid’ to Ukraine as it fights invading Russian troops [2022-02-27T05:26:57Z],0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Environmental policies are taking a back seat in the SA election. Should voters be worried? [2022-03-18T01:19:56Z],0.029218,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.047487,0.0,0.0,0.0,0.0,0.0,0.0,0.0


#### Update the terms matrix

In [173]:
for idx in terms_df.index:
    tfidf = dict(tfidf_df.loc[idx].sort_values(ascending=False).head(10))
    #print(counts)
    terms_df.at[idx,'tfidf'] = list(tfidf.keys()) 

terms_df

,count,tfidf,lda,nmf
Queensland budget 2024: the biggest winners and losers [2024-06-11T04:38:46Z],"[queensland, include, public, spend, election,...","[budget, state, miles, queensland, dick, gover...",NaN,NaN
"Queensland state budget 2024: 50c fares and $1,000 energy rebates favoured over surplus in pre-election spend [2024-06-11T04:00:44Z]","[dick, measure, cost, deficit, queensland, mak...","[dick, measures, deficit, construction, royalt...",NaN,NaN
Queensland officially drought-free for first time since 2013 [2024-05-14T15:00:24Z],"[drought, queensland, ’, rain, country, farmer...","[drought, rains, farmers, rainfall, landscape,...",NaN,NaN
"With discounts galore, Labor’s 2024 Queensland budget has all the subtlety of a ‘going out of business’ sale [2024-06-11T07:00:37Z]","[election, discount, dick, cost, living, measu...","[dick, discounts, polling, election, finished,...",NaN,NaN
CFMEU administrator moves for ‘clean sweep’ of union super fund directors at Cbus [2024-08-28T08:51:22Z],"[union, construction, cfmeu, industry, secreta...","[cfmeu, union, construction, secretary, cbus, ...",NaN,NaN
...,...,...,...,...
Most of Australia’s nuclear waste comes from Lucas Heights – should it stay there? [2022-10-16T16:30:40Z],"[waste, facility, nuclear, estimate, storage, ...","[waste, facility, nuclear, cubic, storage, say...",NaN,NaN
Lidia Thorpe quits Greens party to pursue black sovereignty [2023-02-06T02:15:16Z],"[thorpe, green, voice, party, bandt, support, ...","[thorpe, voice, greens, bandt, referendum, sov...",NaN,NaN
Australia to provide ‘lethal aid’ to Ukraine as it fights invading Russian troops [2022-02-27T05:26:57Z],"[ukraine, russia, russian, china, support, min...","[ukraine, russian, russia, china, wong, lethal...",NaN,NaN
Environmental policies are taking a back seat in the SA election. Should voters be worried? [2022-03-18T01:19:56Z],"[party, south, election, liberal, climate, ene...","[liberal, says, party, election, climate, envi...",NaN,NaN


#### Compare approaches

In [174]:
# Sample 5 random articles
samples = random.sample(range(0,len(terms_df)),5)

for sample in samples:
    doc = terms_df.iloc[sample]
    print(f"[{sample}] {doc.name}")
    print("\t- Counts:\t",doc['count'])
    print("\t- TFIDF:\t",doc['tfidf'])
    print()

[120] Former Greens councillor demands apology as Queensland police again drop charges [2023-04-25T15:00:37Z]
	- Counts:	 ['police', 'protest', 'queensland', 'councillor', 'arrest', 'charge', 'time', 'brisbane', 'service', 'officer']
	- TFIDF:	 ['police', 'protest', 'councillor', 'ethical', 'arrested', 'dropped', 'trespass', 'berkman', 'charges', 'brisbane']

[381] Back to the future with ABC’s reshuffle as content once again crowned king | Weekly Beast [2023-05-12T04:48:14Z]
	- Counts:	 ['news', 'abc', 'report', 'radio', 'content', 'tell', 'email', 'help', '   ', 'hadley']
	- TFIDF:	 ['radio', 'hadley', 'abc', 'humphries', 'content', 'pillars', 'au', 'news', 'barry', 'entertainment']

[350] Girls and women in umpiring allege sexual harassment in AFL-funded study [2022-05-02T02:50:05Z]
	- Counts:	 ['umpire', 'woman', 'girl', 'report', 'afl', 'game', 'make', 'include', 'participation', 'fellow']
	- TFIDF:	 ['girls', 'women', 'umpire', 'report', 'afl', 'coaches', 'participation', 'fellow

### Topic modelling with Latent Dirichlet Allocation (LDA)

[LDA](https://en.wikipedia.org/wiki/Latent_Dirichlet_allocation) is an algorithm for obtaining *topics* (a list of terms) from a document-term matrix. It is a generative probabilistic approach to *decomposition* of the document-term matrix into 2 factor matrices: document-topic and topic-term.

![img](https://editor.analyticsvidhya.com/uploads/26864dtm.JPG)

*Source: [Analytics Vidhya](https://www.analyticsvidhya.com/blog/2021/06/part-2-topic-modeling-and-latent-dirichlet-allocation-lda-using-gensim-and-sklearn/)*

The LDA model requires the number of topics to be set in advance. As it is a generative model, it also runs over a number of iterations. These values usually need to be experimented with to obtain quality topics.

In [175]:
# Set number of topics
num_topics = 20
# Set max number of iteractions
max_iterations = 50

# Create the model
lda_model = LatentDirichletAllocation(n_components=num_topics,max_iter=max_iterations,learning_method='online')

# Fit the model to the data, and use the model to transform the data (do the decomposition)
doc_topic_matrix = lda_model.fit_transform(count_dt_matrix) # Note that this uses the counts already processed

# Obtain the topics
topic_term_matrix = lda_model.components_

#### View the topics

In [190]:
# Get the topics and their terms
lda_topic_dict = {}
for index, topic in enumerate(topic_term_matrix):
    zipped = zip(feature_names, topic)
    top_terms=dict(sorted(zipped, key = lambda t: t[1], reverse=True)[:10])
    #print(top_terms)
    top_terms_list= {key : round(top_terms[key], 4) for key in top_terms.keys()}
    lda_topic_dict[f"topic_{index}"] = top_terms_list

# Print the topics with their terms    
for k,v in lda_topic_dict.items():
    print(k)
    print(v)
    print()

topic_0
{'cost': 330.9651, 'tax': 312.5768, 'price': 292.6083, 'rate': 251.8225, 'inflation': 215.4341, 'high': 192.3854, 'chalmer': 168.6482, 'rise': 167.3933, 'wage': 139.1339, 'income': 135.2683}

topic_1
{'game': 0.0502, 'gambling': 0.0501, 'people': 0.0501, 'national': 0.0501, 'make': 0.0501, 'work': 0.0501, 'sex': 0.0501, 'violence': 0.0501, 'play': 0.0501, 'party': 0.0501}

topic_2
{'dutton': 0.0503, 'defence': 0.0502, 'moon': 0.0502, 'include': 0.0502, 'news': 0.0502, 'minister': 0.0502, 'people': 0.0502, 'cost': 0.0502, 'make': 0.0502, 'morning': 0.0502}

topic_3
{'defamation': 3.3483, 'exclusive': 2.17, 'yacht': 1.779, 'daylight': 1.7772, 'diver': 1.777, 'sicily': 1.7589, 'sook': 1.7588, 'convention': 1.7028, 'tate': 1.6937, 'raid': 1.6271}

topic_4
{'people': 408.8772, 'police': 388.3216, 'health': 334.097, 'service': 294.4313, 'report': 281.048, ' ': 270.8347, 'guardian': 251.768, 'queensland': 250.4551, 'include': 248.3037, '   ': 219.1033}

topic_5
{'school': 0.0505, 'pub

#### List of topics for each document

In [177]:
doc_topic_matrix[0]

array([4.16413413e-01, 1.03734440e-04, 1.03734440e-04, 1.03734440e-04,
       1.32259239e-01, 1.03734440e-04, 1.03734440e-04, 1.03734442e-04,
       1.03734440e-04, 1.03734442e-04, 1.03734441e-04, 1.03734440e-04,
       1.03734440e-04, 1.03734441e-04, 1.42519165e-02, 1.03734440e-04,
       1.03734440e-04, 4.35415680e-01, 1.03734440e-04, 1.03734442e-04])

In [178]:
#doc_topic_matrix[200]

#### Update the terms matrix

In [179]:
for idx,topic in enumerate(doc_topic_matrix):
     # enumerate just takes a list and returns the value and a counter for the loop starting from 0
    topic_num = topic.argmax()
    top_topic = lda_topic_dict[f"topic_{topic_num}"]
    terms_df['lda'].iloc[idx] = list(top_topic.keys())

terms_df

C:\Users\Morgan\AppData\Local\Temp\ipykernel_26168\474454722.py:5: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  terms_df['lda'].iloc[idx] = list(top_topic.keys())


,count,tfidf,lda,nmf
Queensland budget 2024: the biggest winners and losers [2024-06-11T04:38:46Z],"[queensland, include, public, spend, election,...","[budget, state, miles, queensland, dick, gover...","[project, minister, fund, include, climate, ma...",NaN
"Queensland state budget 2024: 50c fares and $1,000 energy rebates favoured over surplus in pre-election spend [2024-06-11T04:00:44Z]","[dick, measure, cost, deficit, queensland, mak...","[dick, measures, deficit, construction, royalt...","[cost, tax, price, rate, inflation, high, chal...",NaN
Queensland officially drought-free for first time since 2013 [2024-05-14T15:00:24Z],"[drought, queensland, ’, rain, country, farmer...","[drought, rains, farmers, rainfall, landscape,...","[project, minister, fund, include, climate, ma...",NaN
"With discounts galore, Labor’s 2024 Queensland budget has all the subtlety of a ‘going out of business’ sale [2024-06-11T07:00:37Z]","[election, discount, dick, cost, living, measu...","[dick, discounts, polling, election, finished,...","[project, minister, fund, include, climate, ma...",NaN
CFMEU administrator moves for ‘clean sweep’ of union super fund directors at Cbus [2024-08-28T08:51:22Z],"[union, construction, cfmeu, industry, secreta...","[cfmeu, union, construction, secretary, cbus, ...","[project, minister, fund, include, climate, ma...",NaN
...,...,...,...,...
Most of Australia’s nuclear waste comes from Lucas Heights – should it stay there? [2022-10-16T16:30:40Z],"[waste, facility, nuclear, estimate, storage, ...","[waste, facility, nuclear, cubic, storage, say...","[project, minister, fund, include, climate, ma...",NaN
Lidia Thorpe quits Greens party to pursue black sovereignty [2023-02-06T02:15:16Z],"[thorpe, green, voice, party, bandt, support, ...","[thorpe, voice, greens, bandt, referendum, sov...","[housing, people, rent, green, aboriginal, fun...",NaN
Australia to provide ‘lethal aid’ to Ukraine as it fights invading Russian troops [2022-02-27T05:26:57Z],"[ukraine, russia, russian, china, support, min...","[ukraine, russian, russia, china, wong, lethal...","[project, minister, fund, include, climate, ma...",NaN
Environmental policies are taking a back seat in the SA election. Should voters be worried? [2022-03-18T01:19:56Z],"[party, south, election, liberal, climate, ene...","[liberal, says, party, election, climate, envi...","[project, minister, fund, include, climate, ma...",NaN


#### Compare approaches

In [180]:
# Sample 5 random articles
samples = random.sample(range(0,len(terms_df)),5)

for sample in samples:
    doc = terms_df.iloc[sample]
    print(f"[{sample}] {doc.name}")
    print("\t- Counts:\t",doc['count'])
    print("\t- TFIDF:\t",doc['tfidf'])
    print("\t- LDA:\t\t",doc['lda'])
    print()

[65] Afternoon Update: Bonza grounded; teenager charged with murdering sister; and Plato’s final hours [2024-04-30T07:03:20Z]
	- Counts:	 ['   ', 'woman', 'man', 'record', 'number', 'old', 'word', 'bonza', 'announce', 'good']
	- TFIDF:	 ['bonza', 'word', 'bear', 'women', 'scroll', 'old', 'detainee', 'allegations', 'stranded', 'busiest']
	- LDA:		 ['people', 'police', 'health', 'service', 'report', ' ', 'guardian', 'queensland', 'include', '   ']

[374] Toondah Harbour developer says project won’t harm bird habitat because species  [2022-10-12T06:32:05Z]
	- Counts:	 ['eastern', 'project', 'habitat', 'population', 'bird', 'impact', 'development', 'specie', 'group', 'walker']
	- TFIDF:	 ['eastern', 'habitat', 'birds', 'moreton', 'harbour', 'feeding', 'walker', 'species', 'bay', 'project']
	- LDA:		 ['project', 'minister', 'fund', 'include', 'climate', 'make', 'election', 'queensland', 'energy', '’']

[225] Japanese-funded $500m project to extract hydrogen from Victorian coal is at risk, s

### Topic modelling with Non-negative Matrix Factorisation (NMF)


[NMF](https://en.wikipedia.org/wiki/Latent_Dirichlet_allocation) is a different algorithm for obtaining *topics* (a list of terms) from a document-term matrix. It also factorises the document-term matrix into 2 factor matrices: document-topic and topic-term.

In [181]:
# Set the number of topics
num_topics = 20

# Create the model
nmf_model = NMF(n_components=num_topics,init='random',beta_loss='frobenius')

# Fit the model to the data and use it to transform the data
doc_topic_nmf = nmf_model.fit_transform(tfidf_dt_matrix)

topic_term_nmf = nmf_model.components_

# THIS IS WHAT I WANT TO OPTIMISE HERE, AN APPROPRIATE CLUSTERING SHOWHING FUNDING

In [182]:
# Get the topics and their terms
nmf_topic_dict = {}
for index, topic in enumerate(topic_term_nmf):
    zipped = zip(feature_names, topic)
    top_terms=dict(sorted(zipped, key = lambda t: t[1], reverse=True)[:10])
    #print(top_terms)
    top_terms_list= {key : round(top_terms[key], 4) for key in top_terms.keys()}
    nmf_topic_dict[f"topic_{index}"] = top_terms_list

# Print the topics with their terms    
for k,v in nmf_topic_dict.items():
    print(k)
    print(v)
    print()



topic_0
{'carcass': 1.6796, 'comeback': 0.7922, 'lag': 0.7177, 'die': 0.4716, 'overfunde': 0.4505, 'dorinda': 0.4189, 'moderate': 0.385, 'frog': 0.3601, 'migrant': 0.3433, 'elicit': 0.3307}

topic_1
{'narrow': 0.4456, 'fulham': 0.243, 'yang': 0.1728, 'pearson': 0.1242, '61': 0.1234, 'analytic': 0.1224, 'security': 0.1215, 'source': 0.1186, 'commence': 0.1144, 'psychology': 0.1072}

topic_2
{'maría': 2.5215, 'untreated': 1.2056, 'unusual': 0.6283, 'laureate': 0.6083, 'retention': 0.5951, 'accountable': 0.5767, 'malice': 0.5753, 'unwilling': 0.485, 'swing': 0.4103, 'primarily': 0.3892}

topic_3
{'julian': 3.1028, 'yang': 0.5842, 'july': 0.5842, 'rebound': 0.575, 'jump': 0.5394, 'continuation': 0.5108, 'rollout': 0.4014, 'possible': 0.3823, 'julie': 0.3823, 'cresta': 0.3765}

topic_4
{'tower': 1.3493, 'moderate': 1.278, 'tracy': 1.2277, 'subpoena': 0.9972, 'leaver': 0.7565, 'theme': 0.6349, 'beauty': 0.5727, 'classify': 0.5678, 'carcass': 0.5414, 'frozen': 0.5319}

topic_5
{'live': 2.1998

In [183]:
 list(top_topic.keys())


['people',
 'police',
 'health',
 'service',
 'report',
 ' ',
 'guardian',
 'queensland',
 'include',
 '   ']

#### Update the terms matrix

In [184]:
for idx,topic in enumerate(doc_topic_nmf):
    topic_num = topic.argmax()
    top_topic = lda_topic_dict[f"topic_{topic_num}"]
    terms_df['nmf'].iloc[idx] = list(top_topic.keys())

terms_df

C:\Users\Morgan\AppData\Local\Temp\ipykernel_26168\2376733025.py:4: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  terms_df['nmf'].iloc[idx] = list(top_topic.keys())


,count,tfidf,lda,nmf
Queensland budget 2024: the biggest winners and losers [2024-06-11T04:38:46Z],"[queensland, include, public, spend, election,...","[budget, state, miles, queensland, dick, gover...","[project, minister, fund, include, climate, ma...","[housing, people, rent, green, aboriginal, fun..."
"Queensland state budget 2024: 50c fares and $1,000 energy rebates favoured over surplus in pre-election spend [2024-06-11T04:00:44Z]","[dick, measure, cost, deficit, queensland, mak...","[dick, measures, deficit, construction, royalt...","[cost, tax, price, rate, inflation, high, chal...","[housing, people, rent, green, aboriginal, fun..."
Queensland officially drought-free for first time since 2013 [2024-05-14T15:00:24Z],"[drought, queensland, ’, rain, country, farmer...","[drought, rains, farmers, rainfall, landscape,...","[project, minister, fund, include, climate, ma...","[bonza, airport, game, airline, nrl, stadium, ..."
"With discounts galore, Labor’s 2024 Queensland budget has all the subtlety of a ‘going out of business’ sale [2024-06-11T07:00:37Z]","[election, discount, dick, cost, living, measu...","[dick, discounts, polling, election, finished,...","[project, minister, fund, include, climate, ma...","[housing, people, rent, green, aboriginal, fun..."
CFMEU administrator moves for ‘clean sweep’ of union super fund directors at Cbus [2024-08-28T08:51:22Z],"[union, construction, cfmeu, industry, secreta...","[cfmeu, union, construction, secretary, cbus, ...","[project, minister, fund, include, climate, ma...","[school, game, student, gambling, , education..."
...,...,...,...,...
Most of Australia’s nuclear waste comes from Lucas Heights – should it stay there? [2022-10-16T16:30:40Z],"[waste, facility, nuclear, estimate, storage, ...","[waste, facility, nuclear, cubic, storage, say...","[project, minister, fund, include, climate, ma...","[game, gambling, people, national, make, work,..."
Lidia Thorpe quits Greens party to pursue black sovereignty [2023-02-06T02:15:16Z],"[thorpe, green, voice, party, bandt, support, ...","[thorpe, voice, greens, bandt, referendum, sov...","[housing, people, rent, green, aboriginal, fun...","[school, game, student, gambling, , education..."
Australia to provide ‘lethal aid’ to Ukraine as it fights invading Russian troops [2022-02-27T05:26:57Z],"[ukraine, russia, russian, china, support, min...","[ukraine, russian, russia, china, wong, lethal...","[project, minister, fund, include, climate, ma...","[project, minister, fund, include, climate, ma..."
Environmental policies are taking a back seat in the SA election. Should voters be worried? [2022-03-18T01:19:56Z],"[party, south, election, liberal, climate, ene...","[liberal, says, party, election, climate, envi...","[project, minister, fund, include, climate, ma...","[cruise, robert, ship, oval, brand, leichhardt..."


### Compare approaches

In [185]:
# Sample 5 random articles
samples = random.sample(range(0,len(terms_df)),5)

for sample in samples:
    doc = terms_df.iloc[sample]
    print(f"[{sample}] {doc.name}")
    print("\t- Counts:\t",doc['count'])
    print("\t- TFIDF:\t",doc['tfidf'])
    print("\t- LDA:\t\t",doc['lda'])
    print("\t- NMF:\t\t",doc['nmf'])
    print()

[65] Afternoon Update: Bonza grounded; teenager charged with murdering sister; and Plato’s final hours [2024-04-30T07:03:20Z]
	- Counts:	 ['   ', 'woman', 'man', 'record', 'number', 'old', 'word', 'bonza', 'announce', 'good']
	- TFIDF:	 ['bonza', 'word', 'bear', 'women', 'scroll', 'old', 'detainee', 'allegations', 'stranded', 'busiest']
	- LDA:		 ['people', 'police', 'health', 'service', 'report', ' ', 'guardian', 'queensland', 'include', '   ']
	- NMF:		 ['flood', 'water', 'nsw', 'council', 'morning', 'queensland', 'emergency', '   ', 'area', 'south']

[39] Queensland government tells landlords to name their price for homes about to lose rental subsidies [2023-10-18T07:02:44Z]
	- Counts:	 ['housing', 'home', 'scanlon', 'nra', 'social', 'buy', 'income', 'affordable', 'queensland', 'sell']
	- TFIDF:	 ['nras', 'scanlon', 'homes', 'housing', 'threshold', 'government', 'affordable', 'expressions', 'social', 'exiting']
	- LDA:		 ['housing', 'people', 'rent', 'green', 'aboriginal', 'fund', '

Sometimes different approaches are used. Sometimes LDA is better, sometimes nmf. LDA/nmf use different. How do we evaluate which is best?

Andrew recommends not just using counts, but it can do pretty good sometimes

In [186]:
print(topic.argmax())


11


In [187]:
# ATTEMPT CLUSTERING
#https://youtu.be/i74DVqMsRWY?list=PL2VXyKi-KpYttggRATQVmgFcQst3z6OlX

km = KMeans(n_clusters=10, random_state=0, n_init="auto")
#km.fit(terms_df.reset_index()["nmf"])
#Fit model
km.fit(doc_topic_nmf)
# At this point it is still on the 10k wide matrix of frequencies, so need to displ

order_centroids = km.cluster_centers_.argsort()[:,::-1]
terms = nmf_model##.get_feature_names_out()

In [188]:
order_centroids

array([[ 2, 16, 14,  0, 11,  4, 13,  1, 17, 10,  7,  9,  5, 19,  3, 18,
        12,  6, 15,  8],
       [17, 12,  0,  6, 16, 19,  1, 15,  9, 13,  4,  2,  5, 10, 14, 18,
         8,  3,  7, 11],
       [ 1,  6,  5, 15, 17,  0,  2, 16, 10,  4,  9, 14, 13,  8, 19,  7,
        18,  3, 12, 11],
       [15,  1, 10, 17, 16,  0,  9,  3,  8, 11, 14,  5, 19,  4,  6,  7,
        12, 13,  2, 18],
       [19, 10,  1, 17,  7, 13, 16, 14,  0,  2, 11,  4,  5,  3, 15,  6,
         9, 18,  8, 12],
       [16, 18,  0, 17, 14, 10,  6,  9, 12,  5,  1, 13,  7, 15,  8,  4,
         2,  3, 19, 11],
       [10, 14,  6, 17,  9,  5,  1,  3, 11,  4, 16,  0, 19, 12,  7, 13,
         8,  2, 15, 18],
       [ 8, 11, 19,  9,  3,  5, 17, 18,  0,  6,  2,  1, 16, 14, 13,  4,
        10, 12, 15,  7],
       [ 0,  4,  7, 10, 16, 13,  2,  1, 17,  9, 14, 15, 12,  5,  3,  6,
         8, 18, 19, 11],
       [12, 13,  7,  0, 10, 11, 16,  4, 14, 17,  1,  5,  2, 19,  9, 15,
         8, 18,  3,  6]], dtype=int64)

In [189]:
#terms_df['nmf_factor'] = pd.factorize(terms_df[['nmf']])[0]